# Senegal Market Access
Using the global friction surface (Weiss et al. 2019) calculate market access to cities > 50,000 in Senegal and within 50km buffer of the country

In [8]:
import sys, os
import rasterio

import numpy as np
import pandas as pd
import geopandas as gpd
import skimage.graph as graph

sys.path.insert(0, r"../../src")

import GOSTnetsraster.market_access as ma

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [2]:
iso3 = 'SEN'
working_folder = "C:/WBG/Work/Projects/SEN_MA"
data_folder = os.path.join(working_folder, "data")
results_folder = os.path.join(working_folder, "results")

for cfolder in [working_folder, data_folder, results_folder]:
    if not os.path.exists(cfolder):
        os.makedirs(cfolder)  

# define input data
sen_friction = os.path.join(data_folder, "SEN_friction_surface.tif")
aoi_file = os.path.join(data_folder, "SEN_AOI_buffer.geojson")
city_file = os.path.join(data_folder, "SEN_cities.geojson")

if not os.path.exists(aoi_file):
    global_admin = r"C:\WBG\Work\data\ADMIN\ADM0.shp"
    all_admin = gpd.read_file(global_admin)
    sen_admin = all_admin.loc[all_admin['ISO_A3'] == iso3]
    sen_admin = sen_admin.to_crs(3857)
    sen_admin['geometry'] = sen_admin.buffer(50000) # 50km buffer
    sen_admin.to_file(aoi_file, driver='GeoJSON')
in_aoi = gpd.read_file(aoi_file)

if not os.path.exists(sen_friction):
    import GOSTrocks.rasterMisc as rMisc
    global_friction = r"C:\WBG\Work\data\FRICTION\2020_motorized_friction_surface.geotiff"
    cur_r = rMisc.clipRaster(rasterio.open(global_friction), in_aoi, sen_friction, crop=False)
in_friction = rasterio.open(sen_friction)
    
if not os.path.exists(city_file):
    global_cities = r"C:/WBG/Work/data/URBAN/GHS_UCDB_GLOBE_R2024A.gpkg"
    all_city = gpd.read_file(global_cities)
    in_aoi = in_aoi.to_crs(all_city.crs)
    sel_city = gpd.sjoin(all_city, in_aoi, how='inner')
    sel_city.loc[:, ['ID_UC_G0','GC_UCN_MAI_2025','GC_CNT_GAD_2025','GC_POP_TOT_2025','geometry']].to_file(city_file, driver='GeoJSON')
dests = gpd.read_file(city_file)     

In [3]:
frictionD = in_friction.read()[0,:,:]
# convert friction surface to traversal time (lazily). Original data are
#    the original data are minutes to travel 1 m, so we will convert to 
#    minutes to cross the cell
frictionD = frictionD * 1000
mcp = graph.MCP_Geometric(frictionD)

In [5]:
# Convert the cities geometry to centroid, then project to crs of friction surface
dests['geometry'] = dests['geometry'].centroid
dests = dests.to_crs(in_friction.crs)

# calculate travel time to nearest cities
travel_costs, traceback = ma.calculate_travel_time(in_friction, mcp, dests)
travel_costs = travel_costs.astype(in_friction.meta['dtype'])
with rasterio.open(os.path.join(results_folder, "least_cost_travel_time_cities.tif"), 'w', **in_friction.meta) as out_file:
    out_file.write_band(1, travel_costs)

C:\Users\WB411133\AppData\Local\Temp\ipykernel_20612\3328050362.py:2: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  dests['geometry'] = dests['geometry'].centroid


In [37]:
# calculate gravity to nearest cities, using the population column GC_POP_TOT_2025
gravity_col = "GC_POP_TOT_2025"
gravity_file = os.path.join(results_folder, f"gravity_cities_{gravity_col}.tif")
gravity = ma.calculate_gravity(in_friction, mcp, dests, gravity_col, outfile = gravity_file)